# Indexing in a ROMS DataArray

The flexibility of `xarray` gives several ways of indexing a DataArray

In [1]:
import xarray as xr
import xroms

In [2]:
# Load a 4D ROMS DataArray
A = xroms.roms_dataset('ocean_avg_0014.nc')
temp = A.temp

In [3]:
### Use indices 

In [4]:
# Choose some index balues
t = 2     # third time frame 
s = -1    # Surface
i = 100   
j = 110

# Numpy type of indexing
temp[t, s, j, i]

<xarray.DataArray 'temp' ()>
array(9.756377, dtype=float32)
Coordinates:
    xi_rho   int64 100
    eta_rho  int64 110
    time     datetime64[ns] 1989-05-30T12:00:00
    s_rho    float64 -0.01562
    z_rho    float32 -0.59467655
    lat_rho  float64 59.7
    lon_rho  float64 1.189
Attributes:
    long_name:  averaged potential temperature
    units:      Celsius
    time:       ocean_time
    field:      temperature, scalar, series

Use the `isel` method to select values by named indices.
This is more readable and independent of the order of the
dimensions in the NetCDF file.

Take `float` of the value to avoid to get the value without
all the extra information.

In [5]:
# Do not need to remember the order of the indices
value = temp.isel(xi_rho=i, eta_rho=j, s_rho=-1, time=2)
print(float(value))

# Compare with numpy style
value = temp[t, s, j, i]
print(float(value))

9.756377220153809
9.756377220153809


### Using coordinate values

The `sel` method can be used to select data based on the coordinate values.

Note that for xi_rho and eta_rho the index equals the coordinate value.

In [6]:
SST = A['temp'].isel(s_rho=-1)   # Easier to use index value for (near) surface

value = SST.sel(xi_rho=i, eta_rho=j, time='1989-05-30 12')
print(float(value))

9.756377220153809


With staggered grid it is difficult to remember if the U point
at the right edge of gridcell (i,j) has index i or i+1.
With coordinates it is easy, it has xi_u = i + 0.5.

In [7]:
u = A.u.isel(time=t, s_rho=s).sel(xi_u=i+0.5, eta_u=j)
print(float(u))

# The grid cell to the right has index = i
u = A.u[t, s, j, i]
print(float(u))

-0.08350111544132233
-0.08350111544132233


For plotting and analyses it is often convenient to work on a *subgrid*.
XROMS has a function for extracting a subgrid and coordinate-based indexing
is useful for indexing correctly.

In [8]:
# Take a subgrid
i0, i1, j0, j1 = 10, 110, 30, 120
B = xroms.subgrid(A, (i0, i1, j0, j1))

In [9]:
# With index values offsets are needed
print(float(A.temp[t, s, j, i]))
print(float(B.temp[t, s, j-j0, i-i0]))

# Indexing by coordinates are invariant under subgrid
print(float(A.temp[t, s].sel(xi_rho=i, eta_rho=j)))
print(float(B.temp[t, s].sel(xi_rho=i, eta_rho=j)))

9.756377220153809
9.756377220153809
9.756377220153809
9.756377220153809


Velocities in a subgrid may or may not be defined
at the edges of the subgrid. This modifies the offset.
By default, the subgrid is *outer*, velocity is included at the edges.

In [10]:
# For velocity it is more complicated
print(float(A.u[t, s, j, i]))          # 
print(float(B.u[t, s, j-j0, i-i0+1]))  # Add 1 since velocity is included at left edge

# Coordinate based velocity works just fine
print(float(A.u[t, s].sel(xi_u=i+0.5, eta_u=j)))
print(float(B.u[t, s].sel(xi_u=i+0.5, eta_u=j)))                        

-0.08350111544132233
-0.08350111544132233
-0.08350111544132233
-0.08350111544132233
